# Build pipeline with registered components

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription - [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace with computer cluster - [Configure workspace](../../configuration.ipynb)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](../../../README.md) - check the getting started section

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Connect to your AML workspace from the Python SDK
- Define `CommandComponent` using YAML, `command_component` decorator
- Create components into workspace
- Create `Pipeline` using registered components.

**Motivations** - This notebook explains different method to create components via SDK then use these components to build pipeline.

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1 Import the required libraries

In [ ]:
# Import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

from azure.ai.ml import MLClient, Input, Output
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import load_component

## 1.2 Configure credential

We are using `DefaultAzureCredential` to get access to workspace. 
`DefaultAzureCredential` should be capable of handling most Azure SDK authentication scenarios. 

Reference for more available credentials if it does not work for you: [configure credential example](../../configuration.ipynb), [azure-identity reference doc](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

In [ ]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

## 1.3 Get a handle to the workspace

We use config file to connect to a workspace. The Azure ML workspace should be configured with computer cluster. [Check this notebook for configure a workspace](../../configuration.ipynb)

In [ ]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

# Retrieve an already attached Azure Machine Learning Compute.
cluster_name = "cpu-cluster"
print(ml_client.compute.get(cluster_name))

# 2. Define and create components into workspace
## 2.1 Load components definition from YAML

In [ ]:
parent_dir = "."
train_model = load_component(path=parent_dir + "/train_model.yml")
score_data = load_component(path=parent_dir + "/score_data.yml")
# print the component as yaml
print(score_data)

## 2.2 Register components into workspace

In [ ]:
# get or create component
try:
    train_model = ml_client.components.get(
        name=train_model.name, version=train_model.version
    )
except:
    train_model = ml_client.components.create_or_update(train_model)

try:
    score_data = ml_client.components.get(
        name=score_data.name, version=score_data.version
    )
except:
    score_data = ml_client.components.create_or_update(score_data)

# print the component as yaml
# NOTE: resources like code, environment are resolved to remote arm id.
print(score_data)

## 2.3 Create components use component decorator

**Note:** component decorated function need to write as separate py file, which will be included in component code snapshot.

In [2]:
%%writefile eval_src/component.py

from mldesigner import command_component, Input, Output

@command_component(
    display_name="Eval Model",
    version="0.0.9",
)
def eval_model(
    scoring_result: Input(type="uri_folder"),
    eval_output: Output(type="uri_folder"),
):
    """A dummy eval component defined by dsl component."""

    from pathlib import Path
    from datetime import datetime
    print ("hello evaluation world...")

    lines = [
        f'Scoring result path: {scoring_result}',
        f'Evaluation output path: {eval_output}',
    ]

    for line in lines:
        print(line)

    # Evaluate the incoming scoring result and output evaluation result.
    # Here only output a dummy file for demo.
    curtime = datetime.now().strftime("%b-%d-%Y %H:%M:%S")
    eval_msg = f"Eval done at {curtime}\n"
    (Path(eval_output) / 'eval_result.txt').write_text(eval_msg)


Overwriting eval_src/component.py


In [ ]:
%load_ext autoreload
%autoreload 2
# in notebook auto reload the component if any code change
from eval_src.component import eval_model

print(type(eval_model))
help(eval_model)

In [ ]:
# create the component if not exists
try:
    eval_model = ml_client.components.get(name="eval_model", version="0.0.9")
except:
    eval_model = ml_client.components.create_or_update(eval_model)

print(eval_model)

# 3. Sample pipeline job
## 3.1 Build pipeline

In [ ]:
# Construct pipeline
@pipeline()
def pipeline_with_registered_components(
    training_input,
    test_input,
    training_max_epochs=20,
    training_learning_rate=1.8,
    learning_rate_schedule="time-based",
):
    """E2E dummy train-score-eval pipeline with registered components."""
    # Call component obj as function: apply given inputs & parameters to create a node in pipeline
    train_with_sample_data = train_model(
        training_data=training_input,
        max_epochs=training_max_epochs,
        learning_rate=training_learning_rate,
        learning_rate_schedule=learning_rate_schedule,
    )

    score_with_sample_data = score_data(
        model_input=train_with_sample_data.outputs.model_output, test_data=test_input
    )
    score_with_sample_data.outputs.score_output.mode = "upload"

    eval_with_sample_data = eval_model(
        scoring_result=score_with_sample_data.outputs.score_output
    )

    # Return: pipeline outputs
    return {
        "trained_model": train_with_sample_data.outputs.model_output,
        "scored_data": score_with_sample_data.outputs.score_output,
        "evaluation_report": eval_with_sample_data.outputs.eval_output,
    }


pipeline_job = pipeline_with_registered_components(
    training_input=Input(type="uri_folder", path=parent_dir + "/data/"),
    test_input=Input(type="uri_folder", path=parent_dir + "/data/"),
    training_max_epochs=20,
    training_learning_rate=1.8,
    learning_rate_schedule="time-based",
)

# set pipeline level compute
pipeline_job.settings.default_compute = "cpu-cluster"

In [ ]:
print(pipeline_job)

## 3.2 Submit pipeline job

In [ ]:
# submit job to workspace
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="pipeline_samples"
)
pipeline_job

In [ ]:
# Wait until the job completes
ml_client.jobs.stream(pipeline_job.name)

# Next Steps
You can see further examples of running a pipeline job [here](../)